In [1]:
import pandas as pd
import re


from sklearn.metrics import accuracy_score
import joblib

In [2]:
columns = ['target', 'id', 'date', 'query', 'user', 'text']

df = pd.read_csv(
    '/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv',
    encoding='latin-1',
    names=columns
)

df.head()

,target,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
len(df)

1600000

In [4]:
def clean_tweet(text):
  text = text.lower()
  text = re.sub(r'http\S+', '', text)
  text = re.sub(r'@\S+', '', text)

  return text.strip()

In [5]:
df['clean_text'] = df['text'].apply(clean_tweet)
df.head()

,target,id,date,query,user,text,clean_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","- awww, that's a bummer. you shoulda got davi..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he can't update his facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball. managed to sa...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","no, it's not behaving at all. i'm mad. why am ..."


In [6]:
df.isna().sum()

target        0
id            0
date          0
query         0
user          0
text          0
clean_text    0
dtype: int64

In [7]:
# Original Dataset
from sklearn.model_selection import train_test_split

def split(df):
    TRAIN_SIZE = 0.70
    TEST_SIZE = 0.15
    VAL_SIZE = 0.15
    
    train_val, test_df = train_test_split(df, test_size=TEST_SIZE, random_state=42, stratify = df['target'])
    
    adjusted_val_size = VAL_SIZE/(TRAIN_SIZE+VAL_SIZE)
    
    train_df, val_df = train_test_split(train_val, test_size=adjusted_val_size, random_state=42, stratify = train_val['target'])
    
    print(f"  Total samples: {len(df):,}")
    print(f"  Training:   {len(train_df):,} ({len(train_df)/len(df)*100:.1f}%)")
    print(f"  Validation: {len(val_df):,} ({len(val_df)/len(df)*100:.1f}%)")
    print(f"  Test:       {len(test_df):,} ({len(test_df)/len(df)*100:.1f}%)")

    return train_df, test_df, val_df

In [8]:
print('Splitting Original Dataset')
train_df, test_df, val_df = split(df)
# print('\nSplitting Scaled Dataset')
# train_df_scaled, test_df_scaled, val_df_scaled = split(df_scaled)

Splitting Original Dataset
  Total samples: 1,600,000
  Training:   1,119,999 (70.0%)
  Validation: 240,001 (15.0%)
  Test:       240,000 (15.0%)


In [9]:
X_train = train_df['clean_text']
y_train = train_df['target']

X_val = val_df['clean_text']
y_val = val_df['target']

# X_train_scaled = train_df_scaled['clean_text']
# y_train_scaled = train_df_scaled['target']

# X_val_scaled = val_df_scaled['clean_text']
# y_val_scaled = val_df_scaled['target']

## Classical ML

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000, n_jobs=-1),
    "Linear SVM": LinearSVC() 
}

vectorizer = TfidfVectorizer(max_features=100000, ngram_range=(1,2))
print("Vectorizing Oringal Dataset")
X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.transform(X_val)

# print("Vectorizing Scaled Dataset")
# X_train_scaled = vectorizer.fit_transform(X_train_scaled)
# X_val_scaled = vectorizer.transform(X_val_scaled)

Vectorizing Oringal Dataset


In [12]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate_metrics(y_val, y_pred, model_name="Model"):
    print(f"\nEvaluation for {model_name}")
    
    acc = accuracy_score(y_val, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_val, y_pred, average='weighted')
    
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    
    # 2. Detailed Report
    print(classification_report(y_val, y_pred))
    cm = confusion_matrix(y_val, y_pred)
    
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall, "cm": cm}

In [13]:
def finetune(models, X_train, y_train, X_val, y_val):
    results = {}
    
    for name, model in models.items():
        print(f"----Training {name}----")
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        metrics = evaluate_metrics(y_val, preds, model_name=name)
        results[name] = metrics
        
    return results

In [14]:
print('---Fine-Tuning On Original Dataset---')
results = finetune(models, X_train, y_train, X_val, y_val)
# print('---Fine-Tuning On Scaled Dataset---')
# results_scaled = finetune(models, X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled)

---Fine-Tuning On Original Dataset---
----Training Naive Bayes----

Evaluation for Naive Bayes
Accuracy:  0.7994
Precision: 0.7995
Recall:    0.7994
F1 Score:  0.7994
              precision    recall  f1-score   support

           0       0.80      0.80      0.80    120001
           4       0.80      0.79      0.80    120000

    accuracy                           0.80    240001
   macro avg       0.80      0.80      0.80    240001
weighted avg       0.80      0.80      0.80    240001

----Training Logistic Regression----

Evaluation for Logistic Regression
Accuracy:  0.8195
Precision: 0.8196
Recall:    0.8195
F1 Score:  0.8194
              precision    recall  f1-score   support

           0       0.83      0.81      0.82    120001
           4       0.81      0.83      0.82    120000

    accuracy                           0.82    240001
   macro avg       0.82      0.82      0.82    240001
weighted avg       0.82      0.82      0.82    240001

----Training Linear SVM----

Evalu

## DistilBert

In [15]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!pip install optimum[onnxruntime]

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 89.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 15.7 MB/s eta 0:00:00


In [16]:
from tqdm.auto import tqdm
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

2026-02-09 17:24:19.549722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770657859.758660      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770657859.816440      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770657860.324664      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770657860.324718      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770657860.324724      55 computation_placer.cc:177] computation placer alr

In [17]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 64
max_length = 64

In [18]:
def clean_tweet_for_bert(text):
    text = str(text).lower()                   
    text = re.sub(r'@[A-Za-z0-9_]+', '', text) 
    text = re.sub(r'http\S+', '', text)        
    text = " ".join(text.split())
    return text.strip()

df['clean_text_bert'] = df['text'].apply(clean_tweet_for_bert)

In [19]:
from datasets import Dataset

def create_hf_dataset(df, text_col='clean_text', target_col='target'):
    target_map = {0: 0, 4: 1}

    df_processed = df.copy()
    df_processed['label'] = df_processed[target_col].map(target_map)
    
    df_processed = df_processed[[text_col, 'label']]
    
    return Dataset.from_pandas(df_processed)

train_dataset = create_hf_dataset(train_df)
val_dataset   = create_hf_dataset(val_df)

# train_dataset_scaled = create_hf_dataset(train_df_scaled)
# val_dataset_scaled   = create_hf_dataset(val_df_scaled)

NameError: name 'train_df_scaled' is not defined

In [24]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def preprocess_function(examples):
    return tokenizer(examples["clean_text"], truncation=True, padding="max_length", max_length=max_length)

print("Tokenizing Original Data...")
encoded_train = train_dataset.map(preprocess_function, batched=True, desc="Tokenizing Train")
encoded_val = val_dataset.map(preprocess_function, batched=True, desc="Tokenizing Val")

# print("Tokenizing Scaled Data...")
# encoded_train_scaled = train_dataset_scaled.map(preprocess_function, batched=True, desc="Tokenizing Train")
# encoded_val_scaled = val_dataset_scaled.map(preprocess_function, batched=True, desc="Tokenizing Val")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenizing Original Data...


Tokenizing Train:   0%|          | 0/1119999 [00:00<?, ? examples/s]

Tokenizing Val:   0%|          | 0/240001 [00:00<?, ? examples/s]

In [20]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

In [21]:
tqdm.pandas() 

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)

args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_steps=50, 
    disable_tqdm=False,
    report_to="none" 
)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
import os
from optimum.onnxruntime import ORTModelForSequenceClassification
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from optimum.onnxruntime import ORTQuantizer

def quantize_to_onnx(input_model_path, output_dir):
    print(f"🚀 Starting conversion for: {input_model_path}")
    
    model_onnx = ORTModelForSequenceClassification.from_pretrained(
        input_model_path,
        export=True
    )

    qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=True)
    
    quantizer = ORTQuantizer.from_pretrained(model_onnx)
    
    quantizer.quantize(
        save_dir=output_dir,
        quantization_config=qconfig
    )
    
    final_file = os.path.join(output_dir, "model_quantized.onnx")
    size_mb = os.path.getsize(final_file) / (1024 * 1024)
    print(f"Model saved to: {final_file}")
    print(f"Final Size: {size_mb:.2f} MB")

Multiple distributions found for package optimum. Picked distribution: optimum-onnx


In [25]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("Training on Original Dataset...")
trainer.train()
trainer.save_model("./distilbert_original")
quantize_to_onnx("./distilbert_original", "./onnx_quantized_original")

/tmp/ipykernel_55/2728500336.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Training on Original Dataset...


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.326500,0.321596,0.860755,0.860735,0.860956,0.860755


`torch_dtype` is deprecated! Use `dtype` instead!


🚀 Starting conversion for: ./distilbert_original


/usr/local/lib/python3.12/dist-packages/transformers/modeling_attn_mask_utils.py:196: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  inverted_mask = torch.tensor(1.0, dtype=dtype) - expanded_mask


Model saved to: ./onnx_quantized_original/model_quantized.onnx
Final Size: 64.45 MB


### MiniLM

In [42]:
minilm_checkpints = 'microsoft/MiniLM-L12-H384-uncased'

tokenizer = AutoTokenizer.from_pretrained(minilm_checkpints)
model = AutoModelForSequenceClassification.from_pretrained(
    minilm_checkpints, 
    num_labels=2,
    id2label={0: "Negative", 1: "Positive"},
    label2id={"Negative": 0, "Positive": 1}
)

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/133M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

In [47]:
args = TrainingArguments(
    output_dir='/minilm',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=10,       
    log_level="info",        
    report_to="none",        
    disable_tqdm=False       
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    compute_metrics=compute_metrics,
)

print("Training MiniLM...")
trainer.train()

The following columns in the Training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, clean_text. If __index_level_0__, clean_text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1,119,999
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Training with DataParallel so batch size has been adjusted to: 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 52,500
  Number of trainable parameters = 33,360,770


Training MiniLM...


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.301100,0.317349,0.863326,0.863309,0.863502,0.863326
2,0.336100,0.306950,0.867921,0.867909,0.868061,0.867921
3,0.297900,0.309074,0.869709,0.869693,0.869892,0.869709


The following columns in the Evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, clean_text. If __index_level_0__, clean_text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 240001
  Batch size = 16
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
The following columns in the Evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, clean_text. If __index_level_0__, clean_text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 240001
  Batch size = 16
/usr/local/

NameError: name 'output_dir' is not defined

In [50]:
print(f"Saving FP32 model to {'./minilm'}...")
trainer.save_model('./minilm')
tokenizer.save_pretrained('./minilm')

Saving model checkpoint to ./minilm
Configuration saved in ./minilm/config.json
Model weights saved in ./minilm/model.safetensors
tokenizer config file saved in ./minilm/tokenizer_config.json
Special tokens file saved in ./minilm/special_tokens_map.json


Saving FP32 model to ./minilm...


('./minilm/tokenizer_config.json',
 './minilm/special_tokens_map.json',
 './minilm/vocab.txt',
 './minilm/added_tokens.json',
 './minilm/tokenizer.json')

In [52]:
import torch
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer

def quantize_native_pytorch(input_path, output_path):
    model = AutoModelForSequenceClassification.from_pretrained(input_path)
    tokenizer = AutoTokenizer.from_pretrained(input_path)

    quantized_model = torch.quantization.quantize_dynamic(
        model, 
        {torch.nn.Linear},  
        dtype=torch.qint8
    )

    os.makedirs(output_path, exist_ok=True)
    
    weights_path = os.path.join(output_path, "quantized_weights.pt")
    torch.save(quantized_model.state_dict(), weights_path)

    model.config.save_pretrained(output_path)
    tokenizer.save_pretrained(output_path)
    
    size_mb = os.path.getsize(weights_path) / (1024 * 1024)
    print(f"\nSuccess! Model reduced to: {size_mb:.2f} MB")
    print(f"Saved to: {output_path}")

quantize_native_pytorch("./minilm", "./minilm_pytorch_quantized")

loading configuration file ./minilm/config.json
Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dtype": "float32",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "id2label": {
    "0": "Negative",
    "1": "Positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "label2id": {
    "Negative": 0,
    "Positive": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "transformers_version": "4.57.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./minilm/model.safetensors
loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map

Loading model from ./minilm...
Quantizing (Int8 Dynamic)...


/tmp/ipykernel_55/754068341.py:15: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  quantized_model = torch.quantization.quantize_dynamic(
Configuration saved in ./minilm_pytorch_quantized/config.json
tokenizer config file saved in ./minilm_pytorch_quantized/tokenizer_config.json
Special tokens file saved in ./minilm_pytorch_quantized/special_tokens_map.json



✅ Success! Model reduced to: 66.22 MB
Saved to: ./minilm_pytorch_quantized


In [27]:
def predict(text, model, tokenizer):
    # Fix 2: Change 'return_tensor' to 'return_tensors'
    inputs = tokenizer(text, return_tensors = 'pt', truncation = True, max_length = 128)

    with torch.no_grad():
        output = model(**inputs)

    logits = output.logits
    probs = torch.nn.functional.softmax(logits, dim=-1)

    pred_index = torch.argmax(probs, dim=-1).item()
    confidence = probs[0][pred_index].item()

    labels = {0: "Negative", 1: "Positive"}

    return labels[pred_index], confidence

In [30]:
import torch

In [32]:
text1 = "Let's put the meme aside, this is actually a banging song"
sentiment, score = predict(text1, model, tokenizer)
print(f"Result: {sentiment} ({score:.1%})")

Result: Negative (68.4%)
Result: Negative (66.3%)
Result: Negative (66.0%)
Result: Negative (67.1%)
Result: Negative (67.3%)
Result: Negative (67.5%)
Result: Negative (65.6%)
Result: Negative (68.4%)


In [51]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="/kaggle/working/minilm", tokenizer="/kaggle/working/minilm")

print(classifier("Let's put the meme aside, this is actually a banging song"))

loading configuration file /kaggle/working/minilm/config.json
Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dtype": "float32",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "id2label": {
    "0": "Negative",
    "1": "Positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "label2id": {
    "Negative": 0,
    "Positive": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "transformers_version": "4.57.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file /kaggle/working/minilm/config.json
Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  

[{'label': 'Positive', 'score': 0.7343416810035706}]
[{'label': 'Positive', 'score': 0.9673196077346802}]
[{'label': 'Positive', 'score': 0.8110005259513855}]
[{'label': 'Positive', 'score': 0.8651001453399658}]
[{'label': 'Negative', 'score': 0.9204580187797546}]
[{'label': 'Positive', 'score': 0.9287347197532654}]
[{'label': 'Positive', 'score': 0.9687601923942566}]
[{'label': 'Negative', 'score': 0.941748321056366}]
